This is based on official [BERTopic](https://maartengr.github.io/BERTopic/index.html) [tutorial](https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing). The details of the algorithm are [here](https://maartengr.github.io/BERTopic/algorithm/algorithm.html). The source code a comprehensive README are [here](https://github.com/MaartenGr/BERTopic).

# **Tutorial** - Topic Modeling with BERTopic
(last updated 01-09-2022)

In this tutorial we will be exploring how to use BERTopic to create topics from the well-known 20Newsgroups dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for.


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [1]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.


In [2]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2023-07-04 07:08:23,570 - BERTopic - Transformed documents to Embeddings
2023-07-04 07:09:03,120 - BERTopic - Reduced dimensionality
2023-07-04 07:09:55,737 - BERTopic - Clustered reduced embeddings


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [3]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6606,-1_to_of_the_and,"[to, of, the, and, is, for, in, you, it, that]",[SECRET PURPOSE OF FALKLANDS WAR; [with IN-VI...
1,0,1833,0_game_team_games_he,"[game, team, games, he, players, season, hocke...",[\nI am trying to think how to respond to this...
2,1,622,1_key_clipper_chip_encryption,"[key, clipper, chip, encryption, keys, governm...",[-----BEGIN PGP SIGNED MESSAGE-----\n\nPlease ...
3,2,526,2_ites_cheek_yep_huh,"[ites, cheek, yep, huh, ken, forget, why, lets...","[\nHuh?, \nYep.\n, ites:]"
4,3,473,3_israel_israeli_jews_arab,"[israel, israeli, jews, arab, jewish, arabs, p...",[Many of you ask me whether I approve of sever...


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [4]:
topic_model.get_topic(0)  # Select the most frequent topic

[('game', 0.010536274383878745),
 ('team', 0.009172789306209189),
 ('games', 0.007302234659191413),
 ('he', 0.007232667852702684),
 ('players', 0.0064170949844346795),
 ('season', 0.006334449924161549),
 ('hockey', 0.0062077937964287845),
 ('play', 0.00586955459529383),
 ('25', 0.005744889812641606),
 ('year', 0.00572572820185558)]

**NOTE**: BERTopic is stochastic, which means that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [5]:
topic_model.topics_[:10]

[0, 4, 58, -1, 79, -1, -1, 0, 0, -1]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topics to get a good
understanding of the topics that were extracted. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [6]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can
be used to understand how confident BERTopic is that certain topics can be found in a document.

To visualize the distributions, we simply call:

In [7]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [8]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [9]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [10]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [11]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update
the topic representation with new parameters for `c-TF-IDF`:


In [12]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [13]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('game', 0.006611181277126179),
 ('team', 0.00564808193199162),
 ('he', 0.005335361843522438),
 ('games', 0.004453640292670838),
 ('the', 0.004083462986489578),
 ('was', 0.0038493641316727513),
 ('players', 0.00381594793637767),
 ('season', 0.003772633046537904),
 ('in', 0.0036866166050752236),
 ('hockey', 0.0036799769261106658)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:





In [14]:
topic_model.reduce_topics(docs, nr_topics=60)

2023-07-03 22:03:51,932 - BERTopic - Reduced number of topics from 214 to 60


In [15]:
# Access the newly updated topics with:
print(topic_model.topics_)

[0, 0, 1, -1, 33, -1, -1, 0, 0, -1, -1, -1, -1, 20, -1, 13, -1, 5, 7, 2, 2, 0, -1, 2, 0, 6, 5, 0, 0, 14, -1, 5, 0, 0, 21, 12, 0, 22, -1, -1, 6, 0, 2, -1, 0, -1, -1, -1, 0, 0, -1, -1, 5, 0, 22, 28, -1, -1, 1, 2, 0, -1, 0, -1, -1, 6, 0, -1, -1, -1, -1, 5, 0, 0, -1, -1, 0, -1, 0, 0, 25, 2, -1, 2, 9, -1, -1, 5, -1, 1, 0, 4, 1, 0, 13, 9, -1, -1, -1, -1, 50, -1, -1, 3, 4, 0, -1, -1, 0, -1, 12, -1, 6, 0, 13, 1, -1, 6, 0, -1, 0, 24, -1, -1, 7, -1, -1, 4, -1, -1, 44, 1, -1, 0, 27, 4, 0, 0, 0, 7, 1, 12, 31, 0, -1, 0, 12, 0, -1, 0, 2, 2, -1, -1, 35, -1, -1, 3, 6, 0, 0, 4, 8, -1, -1, -1, 1, 6, -1, 21, 9, -1, 0, -1, -1, -1, 0, -1, -1, 0, 2, 0, 17, -1, -1, 4, -1, 10, 13, -1, 0, -1, 4, 1, 19, 1, 13, -1, -1, 0, 1, -1, -1, -1, -1, 2, 0, -1, 0, 1, -1, 16, -1, 0, 2, 5, -1, -1, 6, 4, 0, 10, -1, 2, 0, -1, 0, -1, -1, 0, -1, 18, -1, -1, 0, -1, -1, 0, 0, 0, 17, 17, 21, -1, -1, 8, 4, 2, 1, 3, 0, -1, 19, 0, -1, 0, 8, -1, -1, 16, 17, -1, 0, -1, 0, -1, 0, 0, -1, 0, 0, -1, 4, 0, -1, -1, -1, -1, 0, 1, 11, 1, -1, 0,

In [20]:
topic_model.visualize_topics()

In [18]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

In [19]:
topic_model.visualize_hierarchy(top_n_topics=50)

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [16]:
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics

[2, 48, 40, 0, 6]

In [17]:
topic_model.get_topic(71)

False

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [ ]:
# Save model
topic_model.save("my_model")

In [ ]:
# Load model
my_model = BERTopic.load("my_model")

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) and [here](https://huggingface.co/sentence-transformers) for lists of supported sentence transformers models. This means you can develop your own models or adapt available models for your data as well.  


See the [changelog](https://colab.research.google.com/drive/1DYifonOz2-_Dw1TuxV0qLLCeYFswpL4_#scrollTo=kl_o68HWtgm4&line=1&uniqifier=1) for a comprehensive list of features. Pay attention to KeyBERTInspired, ChatGPT, Online/incremental topic modeling, ...